In [1]:
#import libraries
import chess
import numpy as np
from Agent import Agent
from Environment import Environement

In [2]:
White = Agent(isWhite = True)
Black = Agent(isWhite = False)

Game1 = Environement(White, Black)
history = Game1.Game()
print(history)
print(Game1.board)

There is a stalemate
['e2e3', 'e7e6', 'd2d3', 'd7d6', 'g1e2', 'd6d5', 'e2d4', 'g8e7', 'd1g4', 'e6e5', 'g4c8', 'b8c6', 'd4e6', 'c6d4', 'f1e2', 'd8c8', 'e1g1', 'd4c6', 'e2d1', 'e8d7', 'd1e2', 'c6d8', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e2d1', 'd7e8', 'd1e2', 'e8d7', 'e